<a href="https://colab.research.google.com/github/Wn013/-tool-Download-Upload-Google-Drive-version-1.5/blob/main/Copy%20File%20Drive%20%20-%20Wn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copy Folder Google Drive to Google Drive - 1TouchPro

In [ ]:
#@title Input (Optimized)
from ipywidgets import widgets
from IPython.display import display # Import display explicitly

# --- Định nghĩa các Widgets ---

# Sử dụng mô tả rõ ràng hơn và có thể thêm tooltip nếu cần
dest_text = widgets.Text(
    description="Thư mục Đích:",
    placeholder='Dán link thư mục Google Drive của bạn',
    style={'description_width': 'initial'}, # Cho phép mô tả dài hơn nếu cần
    layout=widgets.Layout(width='90%') # Chiều rộng lớn hơn
)

source_text = widgets.Text(
    description="Thư mục Nguồn:",
    placeholder='Dán link thư mục Google Drive nguồn (có thể là shared)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%')
)

# Sử dụng IntText cho các giá trị số nguyên như số trang
# Giá trị 0 có nghĩa là không giới hạn (hoặc từ đầu/đến cuối)
from_page_text = widgets.IntText(
    description="Từ trang (0=từ đầu):",
    value=0, # Giá trị mặc định là số 0
    style={'description_width': 'initial'}
)

to_page_text = widgets.IntText(
    description="Đến trang (0=đến cuối):",
    value=0, # Giá trị mặc định là số 0
    style={'description_width': 'initial'}
)

# Sử dụng FloatText cho dung lượng để cho phép số lẻ (ví dụ: 1.5 GB)
# Lưu ý: Code xử lý chính sau này đã đặt cứng giới hạn là 1024 GB (1TB)
# Widget này có thể không còn tác dụng nếu bạn không sửa lại phần Main.
max_download_size_text = widgets.FloatText(
    description="Tổng dung lượng tối đa(GB) (0=không giới hạn):",
    value=0, # Đặt mặc định là 0 (không giới hạn) thay vì số lớn
    style={'description_width': 'initial'}
    # Bạn có thể bỏ widget này nếu code sau luôn đặt cứng là 1TB
)

exclude_str_text = widgets.Text(
    description="Loại trừ file/folder chứa:",
    placeholder='Phân cách bởi dấu phẩy, ví dụ: .zip, Thư mục tạm',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%')
)


# --- Hiển thị Widgets ---

# Sử dụng VBox để sắp xếp các widget theo chiều dọc cho gọn gàng
print("Vui lòng nhập thông tin:")
input_widgets = widgets.VBox([
    dest_text,
    source_text,
    from_page_text,
    to_page_text,
    max_download_size_text, # Cân nhắc ẩn hoặc xóa nếu không dùng
    exclude_str_text
])

display(input_widgets)

In [ ]:
# @title Run (Optimized Version - Skip Non-Retryable 403)
import os
import time
import re
import sys
import typing as t
import math
# *** THÊM random VÀO IMPORT NẾU CHƯA CÓ Ở ĐẦU FILE ***
import random
# *** THÊM json VÀO IMPORT ĐỂ PARSE ERROR CONTENT NẾU CẦN ***
# import json # Có thể không cần nếu error.reason hoạt động tốt
from googleapiclient.discovery import build, Resource
from googleapiclient.errors import HttpError
from google.colab import auth
from google.colab import drive
from tqdm.notebook import tqdm

# --- Constants ---
FOLDER_MIME_TYPE = 'application/vnd.google-apps.folder'
MAX_RETRIES = 5 # Số lần thử lại tối đa cho các lỗi CÓ THỂ retry
INITIAL_RETRY_DELAY = 1.0 # Thời gian chờ ban đầu (giây)

class DownloadFromDrive:
    # Sử dụng type hints để rõ ràng hơn về kiểu dữ liệu
    def __init__(self):
        self._total_size: float = 0.0 # Tổng kích thước đã copy (MB)
        self._limit_size: float = 0.0 # Giới hạn kích thước (GB) - sẽ được đặt ở phần Main
        self.excluded_strings: t.List[str] = []

    def get_user_credential(self) -> t.Optional[Resource]:
        """Xác thực người dùng và trả về đối tượng Drive service."""
        try:
            auth.authenticate_user()
            drive_service: Resource = build('drive', 'v3', cache_discovery=False)
            return drive_service
        except Exception as e:
            print(f"Lỗi khi xác thực hoặc tạo Drive service: {e}")
            return None

    def get_childs_from_folder(self, drive_service: Resource, folder_id: str, from_page: int, to_page: int) -> t.List[t.Dict[str, t.Any]]:
        """Lấy danh sách các mục con từ một thư mục, hỗ trợ phân trang và loại trừ."""
        # ... (Giữ nguyên code của hàm này) ...
        files: t.List[t.Dict[str, t.Any]] = []
        page_token: t.Optional[str] = None
        query = f"'{folder_id}' in parents and trashed = false"
        if self.excluded_strings:
            not_contains_query = " and ".join([f"not name contains '{ext}'" for ext in self.excluded_strings])
            query += f" and {not_contains_query}"

        pages = 0
        print(f"Fetching file list from folder ID: {folder_id}...")
        while True:
            try:
                pages += 1
                response = drive_service.files().list(
                    q=query,
                    orderBy='name, createdTime',
                    fields='files(id, name, mimeType, size, capabilities(canDownload, canCopy)), nextPageToken', # Thêm capabilities
                    pageToken=page_token,
                    supportsAllDrives=True,
                    includeItemsFromAllDrives=True,
                    pageSize=1000
                ).execute()

                current_files = response.get('files', [])
                print(f"Page {pages}: Found {len(current_files)} items.")
                if to_page == 0 or (from_page <= pages <= to_page):
                    files.extend(current_files)

                page_token = response.get('nextPageToken', None)
                if page_token is None or (to_page > 0 and pages >= to_page):
                    print("Reached end of file list or target page.")
                    break
            except HttpError as error:
                print(f"An HTTP error occurred while listing files (Page {pages}): {error}")
                break # Thoát vòng lặp nếu có lỗi khi list file
            except Exception as e:
                print(f"An unexpected error occurred while listing files (Page {pages}): {str(e)}")
                break # Thoát vòng lặp
        print(f"Total files/folders fetched in specified range: {len(files)}")
        return files


    # --- HÀM SAO CHÉP VỚI RETRY LOGIC - ĐÃ SỬA ĐỔI ĐỂ BỎ QUA LỖI 403 KHÔNG THỂ RETRY ---
    def _execute_with_retry(self, api_call: t.Callable[[], t.Any]) -> t.Optional[t.Dict[str, t.Any]]:
        """Thực thi API call với retry. Bỏ qua ngay lỗi 403 không thể retry (quyền, policy...)."""
        retry_delay = INITIAL_RETRY_DELAY
        for attempt in range(MAX_RETRIES):
            try:
                return api_call() # Thực thi hàm API
            except HttpError as error:
                status_code = error.resp.status
                # Cố gắng lấy 'reason' từ lỗi, nếu không có thì là None
                reason = getattr(error, 'reason', None)
                # Đôi khi reason nằm sâu hơn trong content dạng JSON
                # if not reason and error.content:
                #    try:
                #        error_details = json.loads(error.content.decode())
                #        if 'error' in error_details and 'errors' in error_details['error']:
                #             reason = error_details['error']['errors'][0].get('reason')
                #    except: # Bỏ qua nếu không parse được JSON
                #        pass

                error_info_str = f"{status_code}{f' ({reason})' if reason else ''}"

                # --- LỖI 403: Kiểm tra lý do ---
                if status_code == 403:
                    # Danh sách các lý do 403 KHÔNG nên thử lại -> Bỏ qua ngay
                    non_retryable_403_reasons = [
                        'insufficientFilePermissions',
                        'insufficientPermissions', # Thêm dạng chung
                        'domainPolicy',
                        'fileOwnerNotMemberOfTeamDrive',
                        'cannotCopyFile', # Lý do tiềm năng cho việc chủ sở hữu chặn copy
                        'forbidden', # Chung chung cho quyền, thường không retry được
                        'dailyLimitExceeded', # Giới hạn dùng hàng ngày (khác rate limit tức thời)
                        'userAccessRevoked',
                        'teamDriveMembershipRequired',
                        'appNotAuthorizedToFile',
                        'sharingNotAllowed', # Thêm lý do về chia sẻ
                        # Thêm các lý do khác nếu gặp trong thực tế
                    ]
                    # Danh sách các lý do 403 NÊN thử lại (chủ yếu là rate limit)
                    retryable_403_reasons = [
                         'userRateLimitExceeded',
                         'rateLimitExceeded',
                         'sharingRateLimitExceeded',
                         # Có thể thêm lý do khác nếu cần, vd: backendError (đôi khi 403)
                    ]

                    if reason in non_retryable_403_reasons:
                        print(f"--- Skipping: Non-retryable 403 error: {error_info_str}.")
                        # In chi tiết lỗi để debug nếu cần
                        # print(f"   Error details: {error}")
                        return None # Bỏ qua ngay
                    elif reason in retryable_403_reasons:
                        # Đây là lỗi rate limit, sẽ thử lại ở dưới
                        print(f"--- API Error ({error_info_str}) on attempt {attempt + 1}/{MAX_RETRIES}. Retrying in {retry_delay:.2f}s...")
                        # Không return None, đi xuống phần retry logic
                    else:
                        # Lý do 403 không xác định -> Bỏ qua cho an toàn
                        print(f"--- Skipping: Uncertain 403 error reason: {error_info_str}.")
                        # print(f"   Error details: {error}")
                        return None # Bỏ qua ngay

                # --- CÁC LỖI KHÁC CÓ THỂ THỬ LẠI (429 - Too Many Requests, 5xx - Server Errors) ---
                elif status_code in [429, 500, 502, 503, 504]:
                     print(f"--- API Error ({error_info_str}) on attempt {attempt + 1}/{MAX_RETRIES}. Retrying in {retry_delay:.2f}s...")
                     # Không return None, đi xuống phần retry logic
                # --- CÁC LỖI KHÁC: Không thử lại (ví dụ: 400 Bad Request, 401 Unauthorized, 404 Not Found...) ---
                else:
                    print(f"--- Skipping: Unrecoverable HTTP error: {error_info_str}.")
                    # print(f"   Error details: {error}")
                    return None # Bỏ qua ngay

                # --- LOGIC THỬ LẠI (Chỉ chạy nếu lỗi thuộc nhóm retryable và chưa return None) ---
                time.sleep(retry_delay)
                retry_delay = min(retry_delay * 2 + (random.random() * 0.5), 60) # Exponential backoff

            except Exception as e:
                # Các lỗi không phải HTTP (vd: network error, etc.)
                print(f"--- Skipping: An unexpected non-HTTP error occurred during API call: {e}")
                import traceback
                # traceback.print_exc() # Bật nếu cần debug sâu
                return None # Bỏ qua

        # Nếu vòng lặp kết thúc mà chưa thành công (đã retry hết số lần cho lỗi retryable)
        print(f"--- Skipping: API call failed after {MAX_RETRIES} attempts (last error was likely retryable: {error_info_str}).")
        return None # Trả về None để báo hiệu thất bại sau khi đã retry

    # ==============================================================
    # HÀM COPY_FILE - Sửa đổi nhỏ để kiểm tra capabilities trước khi copy
    # ==============================================================
    def copy_file(self, drive_service: Resource, dest_folder_id: str, source_file: t.Dict[str, t.Any]):
        """Sao chép một tệp hoặc xử lý thư mục con (đệ quy). Bỏ qua file bị chặn copy."""
        file_name = source_file.get('name', 'Unknown File')
        file_id = source_file.get('id')
        file_mime_type = source_file.get('mimeType')
        capabilities = source_file.get('capabilities', {}) # Lấy capabilities

        if not file_id:
            print(f"--- Skipping item [{file_name}] - Missing file ID.")
            return

        # --- Xử lý trường hợp là FILE ---
        if file_mime_type != FOLDER_MIME_TYPE:
            # *** KIỂM TRA TRƯỚC KHI COPY: Kiểm tra quyền copy/download ***
            can_copy = capabilities.get('canCopy', True) # Mặc định là True nếu không có thông tin
            can_download = capabilities.get('canDownload', True) # Mặc định là True

            # Nếu không thể copy HOẶC không thể download (thường đi đôi với nhau khi bị chặn)
            if not can_copy or not can_download:
                 print(f"--- Skipping file [{file_name}] (ID: {file_id}): Copying/Downloading is disabled by owner or policy.")
                 return # Bỏ qua file này

            # Kiểm tra file tồn tại chính xác bằng tên và mimeType
            if not self.check_if_exists(drive_service, dest_folder_id, file_name, file_mime_type):
                print(f"Attempting to copy: [{file_name}] (ID: {file_id})")
                body_file_inf = {'parents': [dest_folder_id], 'name': file_name}

                # Định nghĩa API call để truyền vào hàm retry
                def copy_api_call():
                    return drive_service.files().copy(
                        body=body_file_inf,
                        fileId=file_id,
                        supportsAllDrives=True,
                        fields='id, name, size'
                    ).execute()

                start_time = time.time()
                # Thực thi copy với retry (hàm retry đã được sửa)
                copied_file_info = self._execute_with_retry(copy_api_call)
                end_time = time.time()

                # Nếu copy thành công (không phải None)
                if copied_file_info:
                    # ... (Phần xử lý size, tốc độ, giới hạn dung lượng giữ nguyên) ...
                    filesize = int(source_file.get('size', 0))
                    if filesize == 0 and copied_file_info.get('size'):
                        filesize = int(copied_file_info.get('size', 0))

                    if filesize > 0:
                        size_mb = filesize / (1024 * 1024)
                        self._total_size += size_mb
                        duration = end_time - start_time
                        speed_mb = size_mb / duration if duration > 0.001 else 0
                        print(f"[{file_name}] copied successfully. Size [{size_mb:0.2f}] MB. Speed [{speed_mb:0.2f}] MB/s")
                    else:
                        print(f"[{file_name}] copied (size 0 or unknown).")

                    if self._limit_size > 0 and self._total_size >= (self._limit_size * 1024):
                        self.on_total_size_exceeded(f"Total size {self._total_size:.2f} MB exceeds limit {self._limit_size} GB. Ending the program.")
                else:
                    # Lỗi đã được xử lý và log trong _execute_with_retry, không cần in thêm ở đây
                    # print(f"--- Failed to copy file [{file_name}] (ID: {file_id}) ...")
                    pass # Đã skip hoặc retry thất bại
            else:
                # File đã tồn tại
                print(f"[{file_name}] exists. Skipping.")

        # --- Xử lý trường hợp là THƯ MỤC ---
        else:
             # ... (Phần xử lý thư mục giữ nguyên) ...
            print(f"\nProcessing folder: [{file_name}] (ID: {file_id})")
            existing_sub_folder_id = self.check_if_exists(drive_service, dest_folder_id, file_name, FOLDER_MIME_TYPE)
            sub_folder_id = existing_sub_folder_id

            if not existing_sub_folder_id:
                print(f"Creating subfolder [{file_name}] in destination.")
                sub_folder_id = self.create_folder(drive_service, dest_folder_id, file_name) # create_folder đã có retry

            if sub_folder_id:
                print(f"Entering folder [{file_name}] (Destination ID: {sub_folder_id})...")
                # Lấy danh sách con từ thư mục nguồn (lấy tất cả con)
                source_files_in_folder = self.get_childs_from_folder(drive_service, file_id, 0, 0)
                if source_files_in_folder:
                    print(f"Found {len(source_files_in_folder)} items inside [{file_name}]. Starting copy...")
                    # Gọi đệ quy để sao chép nội dung bên trong
                    self.copy_multiple_files(drive_service, sub_folder_id, source_files_in_folder)
                    print(f"Finished copying contents of folder [{file_name}].\n")
                else:
                     print(f"Source folder [{file_name}] is empty. Nothing to copy inside.")
            else:
                # Lỗi đã được log trong create_folder
                print(f"Failed to create or find destination subfolder [{file_name}]. Skipping contents.")


    # ==============================================================
    # KẾT THÚC HÀM COPY_FILE ĐÃ SỬA
    # ==============================================================

    # ... (Các hàm khác: create_folder, check_if_exists, copy_multiple_files, extract_folder_id_from_url, on_total_size_exceeded, copy_drive_to_drive giữ nguyên) ...

    def create_folder(self, drive_service: Resource, dest_folder_id: str, sub_folder_name: str) -> t.Optional[str]:
        """Tạo thư mục, kiểm tra tồn tại trước, có retry."""
        exist_folder_id = self.check_if_exists(drive_service, dest_folder_id, sub_folder_name, FOLDER_MIME_TYPE)
        if exist_folder_id:
             print(f"Folder '{sub_folder_name}' already exists with ID: {exist_folder_id}. Using existing.")
             return exist_folder_id
        else:
            print(f"Attempting to create folder: {sub_folder_name}")
            sub_folder_inf = {'name': sub_folder_name, 'mimeType': FOLDER_MIME_TYPE, 'parents': [dest_folder_id]}
            def create_api_call():
                return drive_service.files().create(
                    body=sub_folder_inf,
                    fields='id',
                    supportsAllDrives=True
                ).execute()
            folder_info = self._execute_with_retry(create_api_call) # Dùng retry đã sửa
            if folder_info and folder_info.get('id'):
                folder_id = folder_info.get('id')
                print(f"Folder '{sub_folder_name}' created successfully with ID: {folder_id}")
                return folder_id
            else:
                 # Lỗi đã được log trong _execute_with_retry
                 # print(f"--- Failed to create folder [{sub_folder_name}]...")
                 return None

    def check_if_exists(self, drive_service: Resource, dest_folder_id: str, name: str, mime_type: t.Optional[str] = None) -> t.Optional[str]:
        """Kiểm tra sự tồn tại của tệp/thư mục với tên chính xác và tùy chọn mimeType."""
        # ... (Giữ nguyên code của hàm này) ...
        try:
            processed_name = name.replace("\\", "\\\\").replace("'", "\\'")
            query = f"'{dest_folder_id}' in parents and name = '{processed_name}' and trashed=false"
            if mime_type:
                query += f" and mimeType = '{mime_type}'"

            results = drive_service.files().list(
                q=query,
                fields='files(id, name)',
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                pageSize=1
            ).execute()
            files = results.get('files', [])
            if files:
                return files[0]['id']
        except HttpError as error:
            if error.resp.status != 404:
                print(f"An HTTP error occurred while checking existence for '{name}': {error}")
        except Exception as e:
            print(f"An unexpected error occurred while checking existence for '{name}': {e}")
        return None

    def copy_multiple_files(self, drive_service: Resource, dest_folder_id: str, source_files: t.List[t.Dict[str, t.Any]]):
        """Sao chép nhiều tệp/thư mục với thanh tiến trình tqdm."""
        # ... (Giữ nguyên code của hàm này) ...
        total_files = len(source_files)
        if total_files == 0:
            print("No items to copy in this batch.")
            return
        for source_file in tqdm(source_files, desc=f"Copying to {dest_folder_id[:10]}...", unit="item", leave=True):
            self.copy_file(drive_service, dest_folder_id, source_file) # Gọi copy_file đã sửa
            # time.sleep(0.05) # Có thể bỏ comment nếu vẫn gặp rate limit

    def extract_folder_id_from_url(self, url: str) -> t.Optional[str]:
        """Trích xuất ID thư mục/tệp từ URL Google Drive."""
        # ... (Giữ nguyên code của hàm này) ...
        patterns = [
            r'/folders/([-\w]{25,})', r'/drive/folders/([-\w]{25,})',
            r'id=([-\w]{25,})', r'/file/d/([-\w]{25,})' ]
        for pattern in patterns:
            match = re.search(pattern, url)
            if match: return match.group(1)
        if '/' in url:
            potential_id = url.split('?')[0].split('/')[-1]
            if len(potential_id) >= 25 and not '.' in potential_id and potential_id != 'edit':
                 return potential_id
        print(f"Warning: Could not automatically extract ID from URL: {url}")
        return None

    def on_total_size_exceeded(self, message: str):
        """Hàm xử lý khi vượt quá giới hạn dung lượng."""
        # ... (Giữ nguyên code của hàm này) ...
        print(f"\n!!! LIMIT EXCEEDED !!!")
        print(message)
        print(f"Total size copied: {self._total_size / 1024:.2f} GB")
        sys.exit(1)

    def copy_drive_to_drive(self, destDriveLink: str, sourceDriveLink: str, from_page: int, to_page: int):
        """Hàm chính điều phối quá trình sao chép."""
        # ... (Giữ nguyên code của hàm này, nó sử dụng các hàm đã sửa ở trên) ...
        print("Starting Drive to Drive copy process...")
        service = self.get_user_credential()
        if not service:
            print("Failed to get Google Drive credentials. Aborting.")
            return

        start_time_total = time.time()
        dest_folder_id_parent = self.extract_folder_id_from_url(destDriveLink)
        source_folder_id = self.extract_folder_id_from_url(sourceDriveLink)

        if not dest_folder_id_parent or not source_folder_id:
            print(f"Error: Could not extract folder IDs from URLs.")
            if not dest_folder_id_parent: print(f"  Check Dest Link: {destDriveLink}")
            if not source_folder_id: print(f"  Check Source Link: {sourceDriveLink}")
            return

        print(f"Source Folder ID: {source_folder_id}")
        print(f"Destination Parent Folder ID: {dest_folder_id_parent}")
        if self._limit_size > 0: print(f"Effective Size Limit: {self._limit_size} GB")
        else: print("Effective Size Limit: Unlimited")
        if self.excluded_strings: print(f"Excluding files containing: {self.excluded_strings}")

        try:
            print(f"Fetching source folder info...")
            def get_source_folder_call():
                 return service.files().get(fileId=source_folder_id, supportsAllDrives=True, fields='id, name').execute()
            source_folder = self._execute_with_retry(get_source_folder_call) # Dùng retry đã sửa

            if not source_folder:
                 print(f"Error: Could not fetch source folder information (ID: {source_folder_id}). Check ID and permissions. Aborting.")
                 return

            source_folder_name = source_folder.get('name', 'Copied_Folder')
            print(f"Source folder name: '{source_folder_name}'")

            print(f"Ensuring main destination folder '{source_folder_name}' exists inside {dest_folder_id_parent}...")
            new_dest_folder_id = self.create_folder(service, dest_folder_id_parent, source_folder_name) # Dùng create_folder đã sửa

            if not new_dest_folder_id:
                 print(f"Error: Failed to create/find main destination folder '{source_folder_name}'. Aborting.")
                 return

            print(f"Destination folder ready: '{source_folder_name}' (ID: {new_dest_folder_id})")

            print(f"Fetching item list from source folder '{source_folder_name}'...")
            source_files = self.get_childs_from_folder(service, source_folder_id, from_page, to_page) # Dùng get_childs_from_folder đã sửa

            if source_files:
                self.copy_multiple_files(service, new_dest_folder_id, source_files) # Dùng copy_multiple_files đã sửa
            else:
                print("No files or folders found in the source directory (within specified page range) to copy.")

        except HttpError as error:
            print(f"\nAn critical HTTP error occurred during the process: {error}")
            if error.resp.status == 404:
                print("Please double-check if the Source or Destination Folder IDs are correct and you have access permissions.")
        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            import traceback
            traceback.print_exc()

        end_time_total = time.time()
        total_duration_seconds = end_time_total - start_time_total
        total_duration_minutes = total_duration_seconds / 60
        size_gb = self._total_size / 1024
        speed_mb_avg = self._total_size / total_duration_seconds if total_duration_seconds > 0.01 else 0

        print("\n================== SUMMARY ==================")
        # ... (Phần summary giữ nguyên) ...
        print(f"Copy process finished.")
        print(f"Total Size Copied: {size_gb:.3f} GB ({self._total_size:.2f} MB)")
        print(f"Total Time: {total_duration_seconds:.2f} seconds ({total_duration_minutes:.2f} minutes)")
        print(f"Average Speed: {speed_mb_avg:.2f} MB/s (based on tracked file sizes)")
        print(f"Destination: Folder '{source_folder_name}' inside parent ID {dest_folder_id_parent}")
        print("===========================================")


# ==============================================================
# PHẦN MAIN ĐỂ CHẠY TRONG COLAB
# ==============================================================
# Nhớ import random ở đầu file
# import random

# --- Phần lấy input từ widgets và khởi tạo ---
# ... (Giữ nguyên) ...
destDriveLink = ""
sourceDriveLink = ""
fromPage = 0
toPage = 0
limit_gb_input = 0.0
exclude_input = ""

if 'dest_text' in locals() and hasattr(locals()['dest_text'], 'value'): destDriveLink = locals()['dest_text'].value
if 'source_text' in locals() and hasattr(locals()['source_text'], 'value'): sourceDriveLink = locals()['source_text'].value
if 'from_page_text' in locals() and hasattr(locals()['from_page_text'], 'value'):
    try: fromPage = int(locals()['from_page_text'].value)
    except (ValueError, TypeError): fromPage = 0
if 'to_page_text' in locals() and hasattr(locals()['to_page_text'], 'value'):
    try: toPage = int(locals()['to_page_text'].value)
    except (ValueError, TypeError): toPage = 0
if 'exclude_str_text' in locals() and hasattr(locals()['exclude_str_text'], 'value'): exclude_input = locals()['exclude_str_text'].value

if 'max_download_size_text' in locals() and hasattr(locals()['max_download_size_text'], 'value'):
    try:
        limit_gb_input = float(locals()['max_download_size_text'].value)
        if limit_gb_input < 0: limit_gb_input = 0
    except (ValueError, TypeError): limit_gb_input = 0
else: limit_gb_input = 0


# --- TẠO INSTANCE VÀ CHẠY ---
downloader = DownloadFromDrive()
downloader._limit_size = limit_gb_input
downloader.excluded_strings = [ext.strip() for ext in exclude_input.split(",") if ext.strip()]

if not destDriveLink or not sourceDriveLink:
    print("\n" + "="*40)
    print("!!! ERROR: Please provide valid Source and Destination Google Drive links.")
    print("="*40 + "\n")
else:
    downloader.copy_drive_to_drive(destDriveLink, sourceDriveLink, fromPage, toPage)